In [3]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import pygeohash as gh

In [4]:
import sys
import os
sys.path.append(os.path.dirname(sys.path[0]))
from python.data_cleaning import prepare_data_for_groupby
from python.grouped_data import data_for_analysis

In [29]:
csv_path = '/home/linnanchen/code/MagicaBleps/UK-road-safety/raw_data'
df = pd.read_csv(os.path.join(csv_path, 'dft-road-casualty-statistics-accident-1979-2021.csv'))
df.head()

/tmp/ipykernel_30436/1450232399.py:2: DtypeWarning: Columns (0,2,15,16,35) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(csv_path, 'dft-road-casualty-statistics-accident-1979-2021.csv'))


,accident_index,accident_year,accident_reference,location_easting_osgr,location_northing_osgr,longitude,latitude,police_force,accident_severity,number_of_vehicles,...,pedestrian_crossing_physical_facilities,light_conditions,weather_conditions,road_surface_conditions,special_conditions_at_site,carriageway_hazards,urban_or_rural_area,did_police_officer_attend_scene_of_accident,trunk_road_flag,lsoa_of_accident_location
0,197901A11AD14,1979,01A11AD14,NaN,NaN,NaN,NaN,1,3,2,...,-1,1,8,1,-1,0,-1,-1,-1,-1
1,197901A1BAW34,1979,01A1BAW34,198460.0,894000.0,NaN,NaN,1,3,1,...,-1,4,8,3,-1,0,-1,-1,-1,-1
2,197901A1BFD77,1979,01A1BFD77,406380.0,307000.0,NaN,NaN,1,3,2,...,-1,4,8,3,-1,0,-1,-1,-1,-1
3,197901A1BGC20,1979,01A1BGC20,281680.0,440000.0,NaN,NaN,1,3,2,...,-1,4,8,3,-1,0,-1,-1,-1,-1
4,197901A1BGF95,1979,01A1BGF95,153960.0,795000.0,NaN,NaN,1,2,2,...,-1,4,3,3,-1,0,-1,-1,-1,-1


In [30]:
df = df[df['accident_year'] >= 1999]
df

,accident_index,accident_year,accident_reference,location_easting_osgr,location_northing_osgr,longitude,latitude,police_force,accident_severity,number_of_vehicles,...,pedestrian_crossing_physical_facilities,light_conditions,weather_conditions,road_surface_conditions,special_conditions_at_site,carriageway_hazards,urban_or_rural_area,did_police_officer_attend_scene_of_accident,trunk_road_flag,lsoa_of_accident_location
4883216,1999010SU0945,1999,010SU0945,519490.0,203300.0,-0.271752,51.715661,1,3,1,...,0,1,4,1,0,0,2,1,1,E01023583
4883217,1999010SU0946,1999,010SU0946,521740.0,201070.0,-0.239977,51.695136,1,3,2,...,0,6,4,1,0,0,2,1,1,E01023584
4883218,1999010SU0947,1999,010SU0947,519610.0,203240.0,-0.270037,51.715096,1,3,2,...,0,4,1,2,0,0,2,1,1,E01023583
4883219,1999010SU0948,1999,010SU0948,520090.0,202830.0,-0.263233,51.711309,1,2,2,...,0,6,1,1,0,0,2,1,1,E01023583
4883220,1999010SU0949,1999,010SU0949,522640.0,200320.0,-0.227225,51.688200,1,3,4,...,0,1,1,2,0,0,2,1,1,E01023584
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8703906,2021991196247,2021,991196247,325546.0,674548.0,-3.194076,55.958152,99,3,2,...,0,1,1,1,0,0,1,1,-1,-1
8703907,2021991196607,2021,991196607,271195.0,558271.0,-4.010304,54.902512,99,3,2,...,0,6,1,1,0,0,2,1,-1,-1
8703908,2021991197944,2021,991197944,357297.0,860768.0,-2.716825,57.634805,99,3,1,...,0,1,1,1,0,0,2,1,-1,-1
8703909,2021991200639,2021,991200639,326936.0,675925.0,-3.172192,55.970735,99,3,1,...,5,1,9,1,0,0,1,2,-1,-1


In [31]:
df_new = prepare_data_for_groupby(df,5)

max_geohash = df_new['geohash'].value_counts().index[0]

In [32]:
df_max_geo = df_new[df_new['geohash'] == max_geohash]
df_max_geo

,accident_year,longitude,latitude,accident_severity,number_of_vehicles,number_of_casualties,date,day_of_week,time,IsWeekend,month,Daytime,geohash
4884693,1999,-0.129387,51.513396,3,1,1,1999-01-02,7,03:40,False,1,night (23-5,gcpvj
4884699,1999,-0.131536,51.510193,3,1,1,1999-01-04,2,03:29,False,1,night (23-5,gcpvj
4884739,1999,-0.131754,51.508398,3,2,1,1999-02-14,1,20:45,True,2,evening (19-23),gcpvj
4884741,1999,-0.131211,51.507580,3,2,1,1999-02-20,7,17:10,False,2,afternoon rush (15-19),gcpvj
4884745,1999,-0.131754,51.508398,3,2,2,1999-02-16,3,22:40,False,2,evening (19-23),gcpvj
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688248,2021,-0.088068,51.513639,3,2,1,2021-06-08,3,07:40,False,6,morning rush (5-10),gcpvj
8688249,2021,-0.090322,51.513505,3,2,1,2021-07-27,3,16:00,False,7,afternoon rush (15-19),gcpvj
8688250,2021,-0.094904,51.511125,2,2,1,2021-07-24,7,14:50,False,7,office hours (10-15),gcpvj
8688252,2021,-0.088995,51.513538,2,1,1,2021-09-16,5,17:50,False,9,afternoon rush (15-19),gcpvj


In [33]:
df_max_geo = data_for_analysis(df_max_geo)
df_max_geo

,Accidents
date,
1999-01-31,159
1999-02-28,140
1999-03-31,168
1999-04-30,162
1999-05-31,161
...,...
2021-08-31,60
2021-09-30,81
2021-10-31,79
